In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
import sys
sys.path.append("../")

In [ ]:
from pathlib import Path
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from tqdm.auto import tqdm

from src.analysis import coherence
from src.analysis.state_space import prepare_state_trajectory, StateSpaceAnalysisSpec
from src.datasets.speech_equivalence import SpeechEquivalenceDataset

In [ ]:
model_dir = "outputs/models/timit/w2v2_6/rnn_8-weightdecay0.01/word_broad_10frames"
output_dir = "."
dataset_path = "outputs/preprocessed_data/timit"
equivalence_path = "outputs/equivalence_datasets/timit/w2v2_6/word_broad_10frames/equivalence.pkl"
hidden_states_path = "outputs/hidden_states/timit/w2v2_6/hidden_states.h5"
state_space_specs_path = "outputs/state_space_specs/timit/w2v2_6/state_space_specs.pkl"
embeddings_path = "outputs/model_embeddings/timit/w2v2_6/rnn_8-weightdecay0.01/word_broad_10frames/embeddings.npy"

metric = "cosine"

# Retain words with N or more instances
retain_n = 10

In [ ]:
with open(embeddings_path, "rb") as f:
    model_representations: np.ndarray = np.load(f)
with open(equivalence_path, "rb") as f:
    equiv_dataset: SpeechEquivalenceDataset = torch.load(f)
with open(state_space_specs_path, "rb") as f:
    state_space_spec: StateSpaceAnalysisSpec = torch.load(f)["word"]
assert state_space_spec.is_compatible_with(model_representations)

In [ ]:
drop_idxs = [idx for idx, target_frames in enumerate(state_space_spec.target_frame_spans)
               if len(target_frames) < retain_n]
state_space_spec = state_space_spec.drop_labels(drop_idxs)

In [ ]:
trajectory = prepare_state_trajectory(model_representations, state_space_spec, pad=np.nan)
lengths = [np.isnan(traj_i[:, :, 0]).argmax(axis=1) for traj_i in trajectory]

In [ ]:
len(trajectory), np.concatenate(lengths).mean()

## Estimate within-word distance

In [ ]:
within_distance, within_distance_offset = \
    coherence.estimate_within_distance(trajectory, lengths, state_space_spec, metric=metric)

In [ ]:
within_distance_df = pd.DataFrame(within_distance, index=pd.Index(state_space_spec.labels, name="word")) \
    .reset_index() \
    .melt(id_vars=["word"], var_name="frame", value_name="distance")

In [ ]:
within_distance_offset_df = pd.DataFrame(within_distance_offset, index=pd.Index(state_space_spec.labels, name="word")) \
    .reset_index() \
    .melt(id_vars=["word"], var_name="frame", value_name="distance")

## Estimate between-word distance

In [ ]:
between_distance, between_distance_offset = \
    coherence.estimate_between_distance(trajectory, lengths, state_space_spec,
                                        metric=metric)

In [ ]:
between_distances_df = pd.DataFrame(np.nanmean(between_distance, axis=-1),
                                    index=pd.Index(state_space_spec.labels, name="word")) \
    .reset_index() \
    .melt(id_vars=["word"], var_name="frame", value_name="distance")

In [ ]:
between_distances_offset_df = pd.DataFrame(np.nanmean(between_distance_offset, axis=-1),
                                    index=pd.Index(state_space_spec.labels, name="word")) \
    .reset_index() \
    .melt(id_vars=["word"], var_name="frame", value_name="distance")

## Together

In [ ]:
merged_df = pd.concat([within_distance_df.assign(type="within"), between_distances_df.assign(type="between")])
merged_df.to_csv(Path(output_dir) / "distances.csv", index=False)
merged_df

In [ ]:
ax = sns.lineplot(data=merged_df.dropna(), x="frame", y="distance", hue="type")
ax.set_title("Representational distance within- and between-word")
ax.set_xlabel("Frames since word onset")
ax.set_ylabel(f"{metric.capitalize()} distance")

In [ ]:
merged_offset_df = pd.concat([within_distance_offset_df.assign(type="within"),
                              between_distances_offset_df.assign(type="between")])
merged_offset_df.to_csv(Path(output_dir) / "distances_aligned_offset.csv", index=False)
merged_offset_df

In [ ]:
ax = sns.lineplot(data=merged_offset_df.dropna(),
                  x="frame", y="distance", hue="type")
ax.set_title("Representational distance within- and between-word")
ax.set_xlabel("Frames before word offset")
ax.set_ylabel(f"{metric.capitalize()} distance")
ax.set_xlim((0, np.percentile(np.concatenate(lengths), 95)))

## Estimate distance by grouping features

### Onset

In [ ]:
onsets = [word[0] for word in state_space_spec.labels]

onset_distance_df, onset_distance_offset_df = coherence.estimate_category_within_between_distance(
    trajectory, lengths, onsets, metric=metric, labels=state_space_spec.labels
)

In [ ]:
onset_distance_df.to_csv(Path(output_dir) / "distances-grouped_onset.csv", index=False)

In [ ]:
ax = sns.lineplot(data=onset_distance_df.dropna(), x="frame", y="distance", hue="type")
ax.set_title("Representational distance by onset match/mismatch")
ax.set_xlabel("Frames since word onset")
ax.set_ylabel(f"{metric.capitalize()} distance")

In [ ]:
onset_distance_offset_df["time"] = (onset_distance_offset_df.frame - 1) / 20
onset_distance_offset_df.to_csv(Path(output_dir) / "distances-grouped_onset_aligned_offset.csv", index=False)

### Offset

In [ ]:
offsets = [word[-1] for word in state_space_spec.labels]

offset_distance_df, offset_distance_offset_df = coherence.estimate_category_within_between_distance(
    trajectory, lengths, offsets, metric=metric, labels=state_space_spec.labels
)

In [ ]:
offset_distance_df["time"] = (offset_distance_df.frame - 1) / 20
offset_distance_df.to_csv(Path(output_dir) / "distances-grouped_offset.csv", index=False)

In [ ]:
offset_distance_offset_df["time"] = (offset_distance_offset_df.frame - 1) / 20
offset_distance_offset_df.to_csv(Path(output_dir) / "distances-grouped_offset_aligned_offset.csv", index=False)

In [ ]:
ax = sns.lineplot(data=offset_distance_offset_df.dropna(), x="frame", y="distance", hue="type")
ax.set_title("Representational distance by offset match/mismatch")
ax.set_xlabel("Frames before word offset")
ax.set_ylabel(f"{metric.capitalize()} distance")

## Model-free exploration

In [ ]:
knn_window_size = 10

In [ ]:
knn_references = np.concatenate([np.nanmean(traj_i[:, :knn_window_size, :], axis=1)  # .reshape((traj_i.shape[0], -1))
                                 for traj_i in trajectory])
knn_reference_ids = np.stack([(state_space_spec.labels[idx], j)
                                    for idx in range(len(trajectory)) for j in range(trajectory[idx].shape[0])])

In [ ]:
knn_instances = np.random.choice(len(knn_references), 10, replace=False)

In [ ]:
assert knn_references.shape[0] == knn_reference_ids.shape[0]

In [ ]:
from scipy.spatial.distance import cdist, pdist, squareform

In [ ]:
for knn_instance in knn_instances:
    ref_embedding = knn_references[knn_instance]
    knn_instance_results = cdist(knn_references, ref_embedding[None, :], metric=metric).ravel()

    print(knn_reference_ids[knn_instance])
    print("Nearest neighbors:")
    print(knn_reference_ids[knn_instance_results.argsort()[1:10]])
    print("Furthest neighbors:")
    print(knn_reference_ids[(-knn_instance_results).argsort()[:10]])
    print()

### RSA, collapsed over instances

In [ ]:
rsa_ids = np.unique(knn_reference_ids[:, 0])
rsa_references = np.stack([np.nanmean(knn_references[knn_reference_ids[:, 0] == rsa_id], axis=0)
                            for rsa_id in rsa_ids])
rsa_distances = squareform(pdist(rsa_references, metric=metric))
rsa_distances = pd.DataFrame(rsa_distances, index=rsa_ids, columns=rsa_ids)

In [ ]:
sns.heatmap(rsa_distances)

In [ ]:
sns.clustermap(rsa_distances)

In [ ]:
# Print closest pairs
rsa_distances.values[np.diag_indices(len(rsa_distances))] = rsa_distances.values.flatten().mean()
closest_pair_idxs = rsa_distances.values.ravel().argsort()
closest_pair_idxs = np.stack(np.unravel_index(closest_pair_idxs, rsa_distances.shape)).T
closest_pair_idxs = closest_pair_idxs[closest_pair_idxs[:, 0] != closest_pair_idxs[:, 1]]

print_n = 50
for x, y in closest_pair_idxs[:print_n]:
    if x < y:
        print(rsa_distances.index[x], rsa_distances.columns[y], rsa_distances.values[x, y])
print("---")
for x, y in closest_pair_idxs[-print_n:]:
    if x > y:
        print(rsa_distances.index[x], rsa_distances.columns[y], rsa_distances.values[x, y])